In [1]:
import lightgbm as lgb
from lightgbm import early_stopping
# pip 安装3.3.2版本 lightgbm，conda无法安装
from lightgbm import log_evaluation
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
# warnings.simplefilter('ignore')

------------------------------------------------------------------------------  
### 标签构造

In [2]:
# ST名单，数据来源：中信证券
ST_1822_path ='F:\\zzqaq\\data\\ST_2018_2022.csv'
ST_1822 = pd.read_csv(ST_1822_path,converters={'证券代码':str})
ST_1822=ST_1822.rename(columns={'证券代码':'code'})
ST_1822['实施日期'] = pd.to_datetime(ST_1822['实施日期'])
ST_1822['year'] = ST_1822['实施日期'].map(lambda x: x.year)
ST_1822.head(2)

,序号,code,证券简称,实施日期,实施前简称,实施后简称,实施ST原因,证监会行业,通达信行业,year
0,1,600255,鑫科材料,2020-12-17,*ST梦舟,*ST鑫科,公司名称变更,有色金属冶炼和压延加工业,小金属,2020
1,2,600595,中孚实业,2020-12-15,ST中孚,*ST中孚,实施退市风险警示,有色金属冶炼和压延加工业,铝,2020


In [3]:
# 词向量地址
text_vecs_path ='F:\\zzqaq\\data\\merged_text_vecs_fin.csv'
text_vecs = pd.read_csv(text_vecs_path,encoding="gb18030",converters={'code':str,'year':str,'board':str})
text_vecs[:2]

,code,year,board,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_758,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767
0,000004,2018,00,-0.710000,0.082070,0.477866,0.057852,-0.391598,-0.628945,0.020472,...,0.712304,0.12474,-0.027108,-0.224301,0.113962,0.478764,-0.452287,0.260276,-0.280824,-0.278076
1,000004,2020,00,0.377358,-0.239015,0.122558,-0.083076,0.086279,-0.328752,-0.090525,...,0.678198,0.22263,0.201045,0.427176,-0.227349,0.221474,-0.725955,-0.090054,-1.034944,0.125335


In [47]:
# 查看2018-2020三年文本数据都有的公司
text_vecs[text_vecs['code'].isin(text_vecs[text_vecs.duplicated(subset='code', 
            keep=False)]['code'].value_counts().loc[lambda x:x >= 3].index.tolist())]

,code,year,board,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_758,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767
2,000005,2018,00,0.153546,0.146314,0.653403,-0.182612,-0.442344,-0.819281,0.224281,...,0.209316,0.469589,0.335137,0.019455,0.124827,-0.051742,0.129000,0.172739,-0.466742,0.056772
3,000005,2019,00,-0.164486,-0.153540,0.580999,-0.760257,-0.369319,-0.639261,0.350885,...,0.636578,0.114794,0.278186,-0.186800,-0.044021,0.163023,-0.143067,0.196029,-0.621489,-0.026232
4,000005,2020,00,0.072023,-0.243873,1.061119,-0.424241,-0.559872,-1.324847,0.250098,...,0.412652,-0.279709,0.613982,0.158602,0.003455,-0.056981,0.067131,0.309546,-0.748447,-0.005117
8,000009,2018,00,0.194270,0.317419,0.467833,0.136295,0.002458,-0.354879,0.285871,...,0.768452,0.631327,-0.098052,0.035810,0.476584,-0.168230,-0.473727,-0.293490,-0.020190,0.045336
9,000009,2019,00,-0.506743,-0.335151,0.372205,0.037561,-0.009658,-0.790892,0.137321,...,0.502444,0.521744,-0.128294,0.032024,0.491967,-0.106203,-0.146614,-0.290389,0.040633,-0.247396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7774,900953,2019,900,-0.074273,0.154457,-0.049879,-0.362485,-0.142183,-1.539993,0.122449,...,0.507472,0.618078,-0.409758,0.301778,0.295780,-0.174441,-0.004591,-0.406221,-0.385360,-0.111705
7775,900953,2020,900,-0.457361,0.008762,0.125318,-0.590961,-0.048645,-1.370572,0.074247,...,0.410419,0.759839,-0.327164,0.243113,0.292231,-0.007384,-0.057134,-0.060181,-0.359031,-0.218272
7778,900957,2018,900,-0.009751,0.340882,0.314657,-0.046491,0.020513,-0.974554,0.794565,...,0.138930,-0.034939,-0.323018,0.301308,-0.500218,0.106356,-0.337570,-0.547514,0.308305,-0.291306
7779,900957,2019,900,-0.302736,-0.353995,0.156870,-0.344918,0.069496,-2.083967,0.322034,...,0.417746,0.788064,-0.513646,0.069699,-0.231261,-0.164663,-0.127987,-0.393006,0.234855,0.378508


用下一年ST情况标记上一年文本特征

In [4]:
# 设置label
ST_1822[['code','year']] = ST_1822[['code','year']].astype(str)
text_vecs[['code','year']] = text_vecs[['code','year']].astype(str)
merged_df = pd.merge(text_vecs, ST_1822[['code','year','证监会行业','通达信行业']], on=['code', 'year'], how='left')
merged_df['label']=0
merged_df.loc[(merged_df['证监会行业'].notnull()), 'label'] = 1
merged_df[:2]

,code,year,board,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767,证监会行业,通达信行业,label
0,000004,2018,00,-0.710000,0.082070,0.477866,0.057852,-0.391598,-0.628945,0.020472,...,-0.224301,0.113962,0.478764,-0.452287,0.260276,-0.280824,-0.278076,NaN,NaN,0
1,000004,2020,00,0.377358,-0.239015,0.122558,-0.083076,0.086279,-0.328752,-0.090525,...,0.427176,-0.227349,0.221474,-0.725955,-0.090054,-1.034944,0.125335,NaN,NaN,0


---
### 标记行业

In [4]:
# 读取股票代码和行业信息文件
industry_class = 'F:\\zzqaq\data\\class.csv'
industry_class = pd.read_csv(industry_class,encoding="gb18030")
industry_class['股票代码'] = industry_class['股票代码'].str.zfill(6)
industry_class.dropna()
industry_class = industry_class.rename(columns={'股票代码':'code'})
industry_class.head(2)

,code,门类名称及代码,行业大类代码,行业大类名称
0,000998,农、林、牧、渔业（A),1,农业
1,002041,农、林、牧、渔业（A),1,农业


In [23]:
# 所有企业一起添加门类
industry_class = industry_class.drop_duplicates(subset=['code'])
merged_ = pd.merge(merged_df, industry_class[['code','门类名称及代码']], on=['code'], how='left')
merged_.head(2)

,code,year,board,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767,证监会行业,通达信行业,label,门类名称及代码
0,000004,2018,00,-0.710000,0.082070,0.477866,0.057852,-0.391598,-0.628945,0.020472,...,0.113962,0.478764,-0.452287,0.260276,-0.280824,-0.278076,NaN,NaN,0,业(I)
1,000004,2020,00,0.377358,-0.239015,0.122558,-0.083076,0.086279,-0.328752,-0.090525,...,-0.227349,0.221474,-0.725955,-0.090054,-1.034944,0.125335,NaN,NaN,0,业(I)


In [29]:
# 补充ST企业的行业。部分股票的门类信息没有，通达信标记了部分ST企业证监会行业，没有的标记了通达信行业

# 依据标记的证监会行业对行业门类做补充
# 行业大类名称、门类名称及代码 做成字典
code_to_name = dict(zip(industry_class['行业大类名称'],industry_class['门类名称及代码']))
code_to_name['综合'] = '综合(S)'

# 行业标签为空的ST企业
na_inds = merged_.loc[(merged_['label'] == 1) & (merged_['门类名称及代码'].isna()), :].index
# 遍历选中的行
for ind in na_inds:
    # 获取证监会行业列的值
    industry = merged_.loc[ind, '证监会行业']
    
    # 如果该行对应的证监会行业存在于字典中，则将字典中对应的行业大类代码填入 industry_code 列
    if industry in code_to_name:
        merged_.loc[ind, '门类名称及代码'] = code_to_name[industry]

# 查看门类仍未添加的ST企业
data_industry_is_null = merged_.loc[(merged_['门类名称及代码'].isnull()), :]
data_industry_is_null[data_industry_is_null.label==1]

,code,year,board,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767,证监会行业,通达信行业,label,门类名称及代码
257,000760,2019,00,-0.385838,-0.224914,0.543027,0.328857,-0.721601,-0.560210,0.053458,...,0.043679,-0.134540,-0.046718,0.203677,-0.322608,-0.119862,未知,汽车配件,1,NaN
431,002018,2018,00,-0.029458,-0.591314,0.346442,0.002669,0.083528,-0.625951,0.071084,...,-0.180483,0.083209,-0.045403,0.070066,0.251090,-0.140583,未知,农药化肥,1,NaN
631,002220,2019,00,0.055909,0.021237,0.283320,0.382264,-0.118014,-1.316733,0.431458,...,0.739622,-0.171910,-0.368505,-0.215277,-0.257885,-0.283743,未知,食品,1,NaN
1043,002604,2018,00,-0.558887,-0.104502,0.696799,0.227140,-0.218612,-1.176189,0.319080,...,0.356409,0.002580,0.142675,-0.174496,-0.712777,-0.135654,未知,食品,1,NaN
1044,002604,2018,00,-0.558887,-0.104502,0.696799,0.227140,-0.218612,-1.176189,0.319080,...,0.356409,0.002580,0.142675,-0.174496,-0.712777,-0.135654,未知,食品,1,NaN
1168,002711,2019,00,-0.565700,-0.475997,0.295927,-0.495703,-0.114398,-0.601202,0.133138,...,-0.136959,-0.010282,0.177272,-0.108367,0.283219,-0.326032,未知,仓储物流,1,NaN
1169,002711,2019,00,-0.565700,-0.475997,0.295927,-0.495703,-0.114398,-0.601202,0.133138,...,-0.136959,-0.010282,0.177272,-0.108367,0.283219,-0.326032,未知,仓储物流,1,NaN
3698,600074,2018,60,-0.032297,-0.485819,0.268846,-0.673033,-0.238817,-1.172077,0.339586,...,0.588184,0.164243,-0.467048,0.427512,-0.239017,-0.012304,未知,IT设备,1,NaN
4103,600247,2018,60,0.251439,0.058928,0.854758,-0.673732,-0.156888,-1.360575,0.310803,...,-0.707697,-0.288759,0.309554,0.148767,0.102129,-0.651534,未知,商贸代理,1,NaN
4104,600247,2019,60,-0.214940,-0.298723,0.887675,-0.521397,-0.454807,-0.909292,0.344794,...,-0.355706,-0.072038,-0.172805,0.431020,-0.003565,-0.432685,未知,商贸代理,1,NaN


In [30]:
# 手动添加ST企业门类
merged_.loc[merged_['通达信行业'].isin([
                         '化学原料和化学制品制造业', '黑色金属冶炼和压延加工业',             
                         '汽车配件','农药化肥','食品','IT设备','通信设备','电气设备'
                                 ]), '门类名称及代码'] = '制造业(C)'
merged_.loc[merged_['通达信行业'] == '仓储物流', '门类名称及代码'] = '邮政业(G)'
merged_.loc[merged_['通达信行业'].isin(['商贸代理','黄金','百货']), '门类名称及代码'] = '批发和零售业(F)'
merged_.loc[merged_['通达信行业'] == '多元金融', '门类名称及代码'] = '金融业(J)'

merged_.loc[merged_['证监会行业'] == '科学研究和技术服务业', '门类名称及代码'] = '务业(M)'

In [32]:
# 再次查看有无缺少行业标签的ST企业
data_industry_is_null = merged_.loc[(merged_['门类名称及代码'].isnull()), :]
data_industry_is_null[data_industry_is_null.label==1]

,code,year,board,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767,证监会行业,通达信行业,label,门类名称及代码


In [34]:
# 非ST企业公司行业标签为空的标记为“X”
merged_.loc[merged_['门类名称及代码'].isna(), '门类名称及代码'] = 'X'

In [36]:
# 保存
merged_.to_csv('F:\\zzqaq\\data\\data_with_label.csv',encoding="gb18030",index = False)

In [80]:
# 读取
data ='F:\\zzqaq\\data\\data_with_label.csv'
data = pd.read_csv(data,encoding="gb18030",converters={'code':str,'year':str,'board':str})
data.head(2)

,code,year,board,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767,证监会行业,通达信行业,label,门类名称及代码
0,000004,2018,00,-0.710000,0.082070,0.477866,0.057852,-0.391598,-0.628945,0.020472,...,0.113962,0.478764,-0.452287,0.260276,-0.280824,-0.278076,NaN,NaN,0,业(I)
1,000004,2020,00,0.377358,-0.239015,0.122558,-0.083076,0.086279,-0.328752,-0.090525,...,-0.227349,0.221474,-0.725955,-0.090054,-1.034944,0.125335,NaN,NaN,0,业(I)


--------
### 特征合并

In [83]:
# 读取文本指标合并，并保存
text_norm ='F:\\zzqaq\\data\\text_norm.csv'
text_norm = pd.read_csv(text_norm,encoding="gb18030",converters={'code':str,'year':str,'board':str})
text_norm.head(2)
data_with_textnorm = pd.merge(data, text_norm, on=['code','year'], how='left')

data_with_textnorm.to_csv('F:\\zzqaq\\data\\data_with_label_industry_norm.csv',index = False)

In [84]:
# 读取
data_with_label_industry_norm ='F:\\zzqaq\\data\\data_with_label_industry_norm.csv'
data_with_label_industry_norm = pd.read_csv(data_with_label_industry_norm,converters={'code':str,'year':str})
data_with_label_industry_norm.head(2)

,code,year,board,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,证监会行业,通达信行业,label,门类名称及代码,word_count,sentence_count,readability,pos_count,neg_count,tone
0,000004,2018,0,-0.710000,0.082070,0.477866,0.057852,-0.391598,-0.628945,0.020472,...,NaN,NaN,0,业(I),3146,59,53.322034,286,69,0.611268
1,000004,2020,0,0.377358,-0.239015,0.122558,-0.083076,0.086279,-0.328752,-0.090525,...,NaN,NaN,0,业(I),7359,162,45.425926,801,206,0.590864


### 依据行业取样

In [86]:
# st行业情况
result = data_with_label_industry_norm.loc[merged_['label'] == 1, '门类名称及代码'].value_counts()
result

制造业(C)                 141
批发和零售业(F)               23
业(I)                    16
电力、热力、燃气及水生产和供应业(D)     11
邮政业(G)                   9
建筑业(E)                   8
采矿业(B)                   5
房地产业(K)                  5
(L)                      4
务业(M)                    4
金融业(J)                   3
农、林、牧、渔业(A)              2
(Q)                      2
综合(S)                    2
业(R)                     2
设施管理业(N)                 1
农、林、牧、渔业（A)              1
Name: 门类名称及代码, dtype: int64

In [95]:
# 过于稀疏的行业并到未知（X）行业
data_with_label_industry_norm.loc[data_with_label_industry_norm['门类名称及代码'].isin(
    ['设施管理业(N)','农、林、牧、渔业（A)']), '门类名称及代码'] = 'X'

In [96]:
# st行业情况
result = data_with_label_industry_norm.loc[merged_['label'] == 1, '门类名称及代码'].value_counts()
result

制造业(C)                 141
批发和零售业(F)               23
业(I)                    16
电力、热力、燃气及水生产和供应业(D)     11
邮政业(G)                   9
建筑业(E)                   8
采矿业(B)                   5
房地产业(K)                  5
务业(M)                    4
(L)                      4
金融业(J)                   3
业(R)                     2
农、林、牧、渔业(A)              2
综合(S)                    2
(Q)                      2
X                        2
Name: 门类名称及代码, dtype: int64

In [99]:
# 编码
cat_f = ['门类名称及代码']
le = LabelEncoder()
encoded_list = le.fit_transform(data_with_label_industry_norm[cat_f])
data_with_label_industry_norm[cat_f] = encoded_list

In [102]:
# 改列名
data_with_label_industry_norm = data_with_label_industry_norm.rename(
columns={'门类名称及代码':'industry_code'})
# 丢弃不用列
data_with_label_industry_norm = data_with_label_industry_norm.drop(
    columns=['证监会行业','通达信行业'])
# 查看
data_with_label_industry_norm.head()

,code,year,board,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_766,feature_767,label,industry_code,word_count,sentence_count,readability,pos_count,neg_count,tone
0,000004,2018,0,-0.710000,0.082070,0.477866,0.057852,-0.391598,-0.628945,0.020472,...,-0.280824,-0.278076,0,3,3146,59,53.322034,286,69,0.611268
1,000004,2020,0,0.377358,-0.239015,0.122558,-0.083076,0.086279,-0.328752,-0.090525,...,-1.034944,0.125335,0,3,7359,162,45.425926,801,206,0.590864
2,000005,2018,0,0.153546,0.146314,0.653403,-0.182612,-0.442344,-0.819281,0.224281,...,-0.466742,0.056772,0,2,6397,136,47.036765,644,108,0.712766
3,000005,2019,0,-0.164486,-0.153540,0.580999,-0.760257,-0.369319,-0.639261,0.350885,...,-0.621489,-0.026232,0,2,5796,129,44.930233,608,111,0.691238
4,000005,2020,0,0.072023,-0.243873,1.061119,-0.424241,-0.559872,-1.324847,0.250098,...,-0.748447,-0.005117,0,2,4798,112,42.839286,474,107,0.631670


In [105]:
# 保存
data_with_label_industry_norm.to_csv('F:\\zzqaq\\data\\data_to_split.csv',index = False)

In [103]:
# 读取
data_to_split ='F:\\zzqaq\\data\\data_to_split.csv'
data_to_split = pd.read_csv(data_to_split,converters={'code':str,'year':str})

# 利用 industry_code 列的分布将数据集拆分为训练集、验证集和测试集
train_val_df, test_df = train_test_split(
    data_to_split, test_size=0.2, random_state=42, stratify=data_to_split['industry_code'])
train_df, val_df = train_test_split(
    train_val_df, test_size=0.2, random_state=42, stratify=train_val_df['industry_code'])